# Preprocessing data in MNE-Python

`
Authors:
Britta Westner, Alexandre Gramfort, Denis A. Engemann
`

## Setup

We start out with loading the packages we need. These include `matplotlib` for plotting, `os` for path management, `numpy` for numerical computations, and of course `mne`.
We also use matplotlib magic to ask for figure to be plotted inline. 

In [ ]:
%matplotlib inline 
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mne

Let's double check your MNE-Python version. This should give back 1.2.2 or 1.2.3

In [ ]:
mne.__version__

We set the log-level of MNE-Python to 'warning' so the output is less verbose:

In [ ]:
mne.set_log_level('warning')

### Help!

Remember, if you need help just ask ... the machine!
Let's see how to get the docstring information for a function - here, the function `pick_types`.

In [ ]:
mne.pick_types?

## Set the path to the data

You should have downloaded the `ds000117-practical` folder. We have to let Python know, where to find this folder on your disk. You will have to adjust the path below to reflect your computer and path structure!
You can print the whole path and check the directory to double check it's correct.

In [ ]:
# Change the following path to where the folder ds000117 is on your disk
data_path = os.path.expanduser("~/Documents/teaching/practical_meeg_2022_data/ds000117")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [ ]:
print(raw_fname)

Use `bash` to verify the path is really there - if this gives an error, chances are you made a typo in the path!

In [ ]:
ls $raw_fname

## Access and read the raw data

In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

In [ ]:
raw

For general info on importing data you can check:
- for MEG: https://mne.tools/stable/auto_tutorials/io/plot_10_reading_meg_data.html
- for EEG: https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html

## Understand your data file


Now let's look at the measurement info. It can give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.

In [ ]:
print(raw.info)

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How many channels do you have for each type of sensors?</li>
    <li>What is the sampling frequency?</li>
    <li>Have the data been filtered?</li>
    <li>What is the frequency of the line noise?</li>
    <li>Is there any bad channel?</li>
    </ul>
</div>

## A closer look at the info dictionary

raw.info is just a dictionary:

In [ ]:
isinstance(raw.info, dict)

So we can access its elements this way:

In [ ]:
raw.info['sfreq']  # Sampling frequency

In [ ]:
raw.info['bads']  # list of marked bad channels

In [ ]:
raw.info['line_freq']

## A closer look at the channels
Next let's see what channels are present. It is available via the `raw.ch_names` attribute.

In [ ]:
type(raw.ch_names)

In [ ]:
raw.ch_names[:10]  # this prints the first ten channels

You can index it as a list

In [ ]:
raw.ch_names[42]

We can also query the channel type of a specific channel:

In [ ]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Channel #75 is of type:', channel_type)  # print this out in a neat way

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Channel #320 is of type:', channel_type)

The info also contains all the details about the sensors (type, locations, coordinate frame etc.) in `chs`:

In [ ]:
len(raw.info['chs'])

In [ ]:
type(raw.info['chs'])

In [ ]:
raw.info['chs'][0]  # check the first channel

In [ ]:
raw.info['chs'][330]

Now that we know that there is EEG and MEG channels in the data, we can plot both separately:

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='grad');

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

## Setting channel types and re-referencing

Some channels are wrongly defined as EEG in the file. 
Two of these are EOG (EEG061 and EEG062) and EEG063 is actually an ECG channel. EEG064 was recording but not connected to anything, so we'll make it `'misc'` instead. 
We will now set the channel types for those wrongly classified channels. This will be useful for automatic artifact rejection.

In [ ]:
raw.set_channel_types?

In [ ]:
raw.set_channel_types({'EEG061': 'eog',  # actually EOG not EEG
                       'EEG062': 'eog',  # actually EOG not EEG
                       'EEG063': 'ecg',  # actually ECG not EEG
                       'EEG064': 'misc'})  # EEG064 free-floating electrode

# we also rename the EOG and ECG channels:
raw.rename_channels({'EEG061': 'EOG061',
                     'EEG062': 'EOG062',
                     'EEG063': 'ECG063'})

In [ ]:
raw.info

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

After we have fixed the channels, we can now compute an average reference for the EEG. 

In [ ]:
# For setting the reference, we have to load the data into memory:
raw.load_data()
print(raw.info['custom_ref_applied'])  # let's see if there is a reference applied

In [ ]:
# now let's re-reference
raw.set_eeg_reference(ref_channels='average', projection=False)
print(raw.info['projs'])  # not added as a projection
print(raw.info['custom_ref_applied'])

## Accessing the data

To access the data just use the `[]` syntax as to access any element of a list, dict etc. Note that `raw[]` returns two things: the data and the times array.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

In [ ]:
times  # always starts at 0 by convention

Note that `raw[]` returns both the data and the times array.

## Resampling the data

We will now change the sampling frequency of the data to speed up the computations.

In [ ]:
raw.load_data()  # load data into memory
raw.resample(300)

And let's remove unecessary channels - some empty stimulus channels, misc. channels, and HPI channels.

In [ ]:
raw.drop_channels?

In [ ]:
to_drop = ['STI201', 'STI301', 'MISC201', 'MISC202', 'MISC203',
           'MISC204', 'MISC205', 'MISC206', 'MISC301', 'MISC302',
           'MISC303', 'MISC304', 'MISC305', 'MISC306', 'CHPI001',
           'CHPI002', 'CHPI003', 'CHPI004', 'CHPI005', 'CHPI006',
           'CHPI007', 'CHPI008', 'CHPI009']

In [ ]:
raw.drop_channels(to_drop)

## Filtering the data and plotting raw data

We want to filter the data between 0 and 40 Hz using a linear-phase finite-impulse response (FIR) filter.

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Which parameters do we have to set to achieve this, based on the docstring of the `filter` method?</li>
    </ul>
</div>


In [ ]:
raw.filter?

To see what effect filtering has for our data, let's quickly look at our data first! For full functionality, we ask matplotlib to show the plot in a separate window.

In [ ]:
%matplotlib qt
raw.plot()

In [ ]:
raw.filter(0, 40)

Now that we filtered our data, let's look at it again. Can you spot the difference?

In [ ]:
raw.plot()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li> Which data changed more due to the filtering: EEG or MEG?</li>
      <li> Can you find reasons why?</li>
      <li>  Do you see any bad channels?</li>
      <li>  Is there any characteristics you can see in the data?</li>
       </ul>
</div>

For more information on visualizing of raw data, see here: 
https://mne.tools/0.16/auto_tutorials/plot_visualize_raw.html


## Look at the event structure of the data

The data has different events, which mark which stimulus was shown to the participants. The event/trigger structure is as follows:
- 5, 6, 7: famous faces
- 13, 14, 15: unfamiliar faces
- 17, 18, 19: scrambled faces

We first look at which events are there:

In [ ]:
events = mne.find_events(raw, stim_channel='STI101', verbose=True)

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>What is the type of the variable events?</li>
    <li>What is the meaning of the 3 columns of events?</li>
    <li>How many events of the value 5 are there?
    </ul>
</div>

 

There was a time offset of 34.5 ms in the stimulus presentation. We need to correct the events accordingly.

In [ ]:
delay = int(round(0.0345 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

Let's visualize the paradigm:

In [ ]:
events = events[events[:, 2] < 20]  # take only events with code less than 20

In [ ]:
fig = mne.viz.plot_events(events, raw.info['sfreq']);

For event trigger and conditions we use a Python dictionary with keys that contain "/" for grouping sub-conditions

In [ ]:
event_id = {
    'face/famous/first': 5,
    'face/famous/immediate': 6,
    'face/famous/long': 7,
    'face/unfamiliar/first': 13,
    'face/unfamiliar/immediate': 14,
    'face/unfamiliar/long': 15,
    'scrambled/first': 17,
    'scrambled/immediate': 18,
    'scrambled/long': 19,
}

In [ ]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

We can now re-visit our raw data plot:

In [ ]:
raw.plot(event_id=event_id, events=events);

## Epoch data and artifact rejection

Define epochs parameters:

In [ ]:
tmin = -0.5  # start of each epoch (500ms before the trigger)
tmax = 2.0  # end of each epoch (2000ms after the trigger)

Define the baseline period:

In [ ]:
baseline = (-0.2, 0)  # means from 200ms before to stim onset (t = 0)

We also pick channels now - MEG, EEG and EOG channels

In [ ]:
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True,
                       stim=False, exclude='bads')

The easiest (and maybe also most dangerous?) way to clean your data is to define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers and EOG.

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>The <a href="https://autoreject.github.io/">autoreject project</a> aims to solve this problem of reject parameter setting. See the <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a> for more info.</li>
    </ul>
</div>

In [ ]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

Now we can put all of this together and create epochs:

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [ ]:
print(epochs)  # let's look at some details about the epochs object

Let's explicitly drop the epochs we identified as _bad_ through the thresholds we identified above:

In [ ]:
epochs.drop_bad()  # remove bad epochs based on reject

In [ ]:
epochs.load_data()  # load data in memory

## A closer look at artifact rejection


First, let's have a closer look at the methods of the epochs object.
Uncomment the line below and hit ``epochs.<TAB>``

In [ ]:
#epochs.

See how epochs were dropped

In [ ]:
%matplotlib inline
epochs.plot_drop_log();

### Wait a second, did we just loose half of our epochs due to EOG???

We can probably do better. Let's use the PCA-based signal space projection (SSP) to regress out spatial patterns related to EOG and other offenders, ie., ECG.

Here is the workflow, we'll first detect EOG artifacts and visualize their impact. Then we'll compute related spatial patterns to mitigate these artifacts.

In [ ]:
# There is a function to create EOG epochs:
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint();

Let's see where those EOG segments show up in our raw data:

In [ ]:
%matplotlib qt
raw.plot(events=eog_epochs.events);

Let's compute SSP projections based on the EOG:

In [ ]:
projs_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)

In [ ]:
projs_eog  # let's check what they look like

In [ ]:
%matplotlib inline
mne.viz.plot_projs_topomap(projs_eog, info=epochs.info);

Now the important question is how many components one should keep? Tip: some of them don't look like clear artifact patterns. 

The good news is that we don't need to decide __*right*__ now - as you could see the projectors are stored with the data, but inactive at the moment.

BUT: let's repeat this procedure for the ECG, i.e. heart beat artifacts

In [ ]:
# same business, same issue for ECG
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

We can see that we also face contamination from the cardiac signal... we'll project that out as well.

In [ ]:
projs_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)
mne.viz.plot_projs_topomap(projs_ecg, info=epochs.info);

## Apply projections and visualize effect

Now let's reverse our previous artifact rejection and apply the projections instead. 

In [ ]:
# we remove the EOG from our rejection here:
reject_no_eog = dict(mag=reject['mag'], grad=reject['grad']) 

epochs_clean = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                          picks=picks, baseline=baseline,
                          preload=False,
                          reject=reject_no_eog)

# and then we add the EOG and ECG projs (but we don't apply them yet!)
epochs_clean.add_proj(projs_eog + projs_ecg)


Let's look at one frontal MEG channel before applying the projections:

In [ ]:
epochs_clean.plot_image(picks='MEG0123', sigma=1.);

Now let's apply the projections to a copy and plot this channel again!

In [ ]:
epochs_proj = epochs_clean.copy().apply_proj()  # apply projs on a copy

epochs_proj.plot_image(picks='MEG0123', sigma=1.);

We established earlier, that probably not all projections capture eye blinks and cardiac artifacts. So let's repeat this procedure but only project out the _first_ projection per channel type!

In [ ]:
epochs_clean.del_proj()
epochs_clean.add_proj(projs_eog[::3] + projs_ecg[::3])  # only add some SSP projs
epochs_proj = epochs_clean.copy().apply_proj()  # apply projs on a copy

epochs_proj.plot_image(picks='MEG0123', sigma=1.);

In [ ]:
epochs_proj.info

This way, we now keep all trials, but remove eye blinks and cardiac artifacts. We will now save the data with the SSP projections _unapplied_.

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>MNE keeps SSP projections inside the info and allows to apply them later.</li>
    </ul>
</div>

#### Some thoughts on artifact rejection

We now tackled the artifacts in this data set by computing SSP projections. There are many other ways to do artifact rejection:

- mark artifacts by hand (visual inspection)
- use thresholds (which failed on this dataset!)
- use ICA
- use an automated pipeline, e.g. the <a href="https://autoreject.github.io/">autoreject project</a>
- ...

The best recommendation is: get to know your (raw) data!

## Save Epochs

In [ ]:
# let's overwrite
epochs = epochs_clean

The standard way is to save the epochs as a `.fif` file together with all the header data. Epochs are saved with the suffix `-epo.fif`.

In [ ]:
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')  # create the file name
epochs_fname

In [ ]:
epochs.save(epochs_fname, overwrite=True) 

## Bonus: Visualizing epochs data

See [this page](https://mne.tools/stable/auto_tutorials/epochs/20_visualize_epochs.html) for options on how to visualize epochs.

In [ ]:
# We have already looked at the epochs in a stacked plot:

epochs_proj.plot_image(picks='EEG065', sigma=1.);

We can also look at the epochs in a data browser window:

In [ ]:
%matplotlib qt
epochs.plot();

In [ ]:
epochs.info